In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('./Top-100 Trending Books.csv')

In [3]:
def extract_asin(url):
    match = re.search(r'/dp/([A-Z0-9]{10})', str(url))
    return match.group(1) if match else ''

def make_cover_url(asin):
    if asin:
        return f'https://images-na.ssl-images-amazon.com/images/P/{asin}.01._SCLZZZZZZZ_.jpg'
    return ''

def remove_surrounding_double_quotes(text):
    if isinstance(text, str) and text.startswith('"') and text.endswith('"'):
        return text[1:-1]
    return text

## Create coverUrl column

In [4]:
df['coverUrl'] = df['url'].apply(lambda url: make_cover_url(extract_asin(url)))

## Remove old url column and adjust order

In [5]:
df = df.drop(columns=['url'])
cols = list(df.columns)

## Move coverUrl to end of row

In [6]:
cols = [col for col in cols if col != 'coverUrl'] + ['coverUrl']
df = df[cols]

In [7]:
df.head()

,Rank,book title,book price,rating,author,year of publication,genre,coverUrl
0,1,"Iron Flame (The Empyrean, 2)",18.42,4.1,Rebecca Yarros,2023,Fantasy Romance,https://images-na.ssl-images-amazon.com/images...
1,2,The Woman in Me,20.93,4.5,Britney Spears,2023,Memoir,https://images-na.ssl-images-amazon.com/images...
2,3,My Name Is Barbra,31.50,4.5,Barbra Streisand,2023,Autobiography,https://images-na.ssl-images-amazon.com/images...
3,4,"Friends, Lovers, and the Big Terrible Thing: A...",23.99,4.4,Matthew Perry,2023,Memoir,https://images-na.ssl-images-amazon.com/images...
4,5,How to Catch a Turkey,5.65,4.8,Adam Wallace,2018,"Childrens, Fiction",https://images-na.ssl-images-amazon.com/images...


In [8]:
reviews = pd.read_csv('customer reviews.csv')
books = df

## Extract ISBN from coverUrl

In [9]:
books['ISBN'] = books['coverUrl'].apply(lambda url: url.split('/')[-1].split('.')[0] if pd.notnull(url) else '')

books['ISBN'] = books['ISBN'].str.zfill(10)
reviews['ASIN'] = reviews['ASIN'].astype(str).str.zfill(10)

merged = pd.merge(
    reviews,
    books,
    left_on='ASIN',
    right_on='ISBN',
    how='left'
)

## Select necessary features

In [10]:
final = merged.rename(columns={
    'book name': 'Book.title',
    'author': 'Book.author',
    'ASIN': 'Book.isbn',
    'coverUrl': 'Book.coverUrl',
    'reviewer': 'Review.reviewerName',
    'reviewer rating': 'Review.rating',
    'review description': 'Review.comment'
})[
    ['Book.title', 'Book.author', 'Book.isbn', 'Book.coverUrl', 'Review.reviewerName', 'Review.rating', 'Review.comment']
]

## Normalize title and review comments

In [11]:
final['Book.title'] = final['Book.title'].apply(remove_surrounding_double_quotes)
final['Review.comment'] = final['Review.comment'].apply(remove_surrounding_double_quotes)

In [14]:
final.to_json('books_reviews_dataset.json', orient='records', force_ascii=False, lines=False)

In [13]:
final.head()


,Book.title,Book.author,Book.isbn,Book.coverUrl,Review.reviewerName,Review.rating,Review.comment
0,The Woman in Me,Britney Spears,1668009048,https://images-na.ssl-images-amazon.com/images...,Murderess Marbie,4,I'm only a third way in. Shipped lightening fa...
1,The Woman in Me,Britney Spears,1668009048,https://images-na.ssl-images-amazon.com/images...,L J,5,There have been so many times when I was scare...
2,The Woman in Me,Britney Spears,1668009048,https://images-na.ssl-images-amazon.com/images...,Jamie,5,The media could not be loaded. I personally ha...
3,The Woman in Me,Britney Spears,1668009048,https://images-na.ssl-images-amazon.com/images...,KMG,5,I have been a fan of Britney's music since the...
4,The Woman in Me,Britney Spears,1668009048,https://images-na.ssl-images-amazon.com/images...,Stephanie Brown,5,"Whether or not you’re a fan, it’s a great read..."
